In [4]:
import os
import sys
import time
import numpy as np
from PIL import Image
import cv2

In [148]:
# ASCII characters used to represent the grayscale intensity
ASCII_CHARS_REV = "@.. "
ASCII_CHARS = ""
for char in reversed(ASCII_CHARS_REV):
    ASCII_CHARS += char

def image_to_ascii(image, width:int=80):
    """
    Convert an image to ASCII characters.

    Params:
        image (ImageFile): PIL Image object
        width (int): Desired output width

    Returns:
        (str): ASCII art as string
    """
    # Calculate aspect ratio and set new height
    aspect_ratio = image.height / image.width
    new_height = int(aspect_ratio * width * 0.5)  # Adjusting for font aspect ratio

    # Resize the image
    image = image.resize((width, new_height))
    
    # Convert to grayscale
    gray_image = image.convert("L")
    # display(gray_image)

    # Convert pixels to ASCII
    pixels = np.array(gray_image)
    max_pixel = max([max(row) for row in pixels])
    min_pixel = min([min(row) for row in pixels])
    sep = int((max_pixel - min_pixel)/len(ASCII_CHARS))
    # print(sep)

    ascii_str = ""
    for row in pixels:
        ascii_str += "".join(ASCII_CHARS[min(pixel // sep, pixel // 65)] for pixel in row) + "\n"

    return ascii_str

In [149]:
gif_path = "../data/thick_hello_world.png"
image = Image.open(gif_path)

In [ ]:
print(
    image_to_ascii(
        image = image
    )
)